In [1]:
!pip install ultralytics
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 10.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-

In [2]:
!ls '/content/DATASET3.zip'
!unzip '/content/DATASET3.zip' -d dataset

/content/DATASET3.zip
Archive:  /content/DATASET3.zip
  inflating: dataset/README.dataset.txt  
  inflating: dataset/README.roboflow.txt  
  inflating: dataset/data.yaml       
   creating: dataset/test/
   creating: dataset/test/images/
 extracting: dataset/test/images/008BE35A-ED85-43FA-BFD5-8344139510A8_jpeg.rf.36908adaf34f6631423c41b6f5fff7db.jpg  
 extracting: dataset/test/images/00BBDD2C-C2A1-463B-8D00-992EB02D6EC2_jpeg.rf.e0c4df7d27de86254dedac365ff1c7d4.jpg  
 extracting: dataset/test/images/01D0AD14-E522-4737-85C9-551BF4FF9D96_jpeg.rf.bff626e61ed25516b6b2de262afd7a19.jpg  
 extracting: dataset/test/images/067ACD18-F150-4040-AC28-5E64B7FA4799_jpeg.rf.77ba8b2ed0c9ccd79398fb07e45d2ca9.jpg  
 extracting: dataset/test/images/0C7149D2-80E6-4D03-96D5-066D8308CA15_jpeg.rf.2f0eac5c8877981a5f5cf1c1d87511fb.jpg  
 extracting: dataset/test/images/0E7C212C-F026-4668-8C36-E59BDCD8A677_jpeg.rf.e3e551560066c23e43c91c2c80dc6884.jpg  
 extracting: dataset/test/images/127E44CB-1D40-4740-B907-90D

In [ ]:
import optuna
from ultralytics import YOLO

def objective(trial):
    # Hiperparámetros que queremos ajustar
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
    img_size = trial.suggest_categorical('img_size', [640, 1024])

    # Carga el modelo
    model = YOLO('yolov8n.pt', device=0)  # Asegúrate de especificar el dispositivo adecuadamente

    # Resultados del entrenamiento
    results = model.train(data='/content/dataset/data.yaml',
                          epochs=50,  # Reducir para pruebas rápidas
                          batch_size=batch_size,
                          imgsz=img_size,
                          lr=lr,
                          device=0)

    # Obtenemos el mejor resultado de precisión de validación
    best_result = results['val/box_loss'].min()  # Asumiendo que queremos minimizar box_loss
    return best_result

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)  # Ejecuta 20 pruebas

print("Best hyperparameters: {}".format(study.best_params))



In [4]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)

# Train the model with 2 GPUs
results = model.train(data='/content/dataset/data.yaml', epochs=200, imgsz=640, device=0)

100%|██████████| 6.23M/6.23M [00:00<00:00, 21.7MB/s]


Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/dataset/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

100%|██████████| 755k/755k [00:00<00:00, 4.70MB/s]


Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytic

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    200/200       2.3G     0.2039     0.1267      0.824          1        640: 100%|██████████| 75/75 [00:23<00:00,  3.17it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:01<00:00,  3.01it/s]


                   all        114        121      0.996      0.935      0.966      0.961

200 epochs completed in 1.583 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:03<00:00,  1.20it/s]


                   all        114        121      0.997      0.935      0.967      0.963
           ALKASELTZER         13         15          1      0.733      0.867      0.859
          ASPIRINA_100         11         11      0.995          1      0.995      0.983
             BUSCAPINA         10         10      0.994          1      0.995      0.995
           DOLEX_GRIPA         13         13          1      0.846      0.923      0.923
              GAVISCON         13         13          1       0.85      0.959      0.949
       IBUFLASH_MIGRAN         12         12      0.995          1      0.995      0.995
     IBUPROFENO_GENFAR         13         13          1          1      0.995      0.987
        LEVONORGESTREL         11         11      0.992          1      0.995      0.995
     NORAVER_GRIPA_DIA         11         11          1          1      0.995      0.995
              VITAC_MK         12         12      0.996      0.917      0.955      0.949
Speed: 0.4ms preproce

In [5]:
# Validate model

from ultralytics import YOLO

# Load a model
model = YOLO('runs/detect/train/weights/best.pt')  # load a custom model

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/dataset/valid/labels.cache... 114 images, 0 backgrounds, 0 corrupt: 100%|██████████| 114/114 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:04<00:00,  1.87it/s]


                   all        114        121      0.997      0.935      0.967      0.962
           ALKASELTZER         13         15          1      0.733      0.867       0.85
          ASPIRINA_100         11         11      0.995          1      0.995      0.986
             BUSCAPINA         10         10      0.994          1      0.995      0.995
           DOLEX_GRIPA         13         13          1      0.846      0.923      0.923
              GAVISCON         13         13          1       0.85      0.959      0.949
       IBUFLASH_MIGRAN         12         12      0.995          1      0.995      0.995
     IBUPROFENO_GENFAR         13         13          1          1      0.995      0.986
        LEVONORGESTREL         11         11      0.992          1      0.995      0.995
     NORAVER_GRIPA_DIA         11         11          1          1      0.995      0.995
              VITAC_MK         12         12      0.996      0.917      0.955      0.949
Speed: 0.7ms preproce

array([    0.84974,      0.9858,       0.995,       0.923,     0.94908,       0.995,     0.98605,       0.995,       0.995,     0.94862])

In [6]:
#  Prediction using trained model
import os
from ultralytics import YOLO

image_dir = '/content/dataset/test/images'

# Check if the directory exists
if os.path.exists(image_dir):
    # Run inference if the directory exists
    model.predict(image_dir, save=True, imgsz=640, conf=0.2)
else:
    print(f"Error: The specified path '{image_dir}' does not exist.")

# Load a pretrained YOLOv8n model
model = YOLO('runs/detect/train/weights/best.pt')

# Run inference
model.predict('/content/dataset/test/images', save=True, imgsz=640, conf=0.2)


image 1/57 /content/dataset/test/images/008BE35A-ED85-43FA-BFD5-8344139510A8_jpeg.rf.36908adaf34f6631423c41b6f5fff7db.jpg: 640x640 1 BUSCAPINA, 7.3ms
image 2/57 /content/dataset/test/images/00BBDD2C-C2A1-463B-8D00-992EB02D6EC2_jpeg.rf.e0c4df7d27de86254dedac365ff1c7d4.jpg: 640x640 1 IBUPROFENO_GENFAR, 7.4ms
image 3/57 /content/dataset/test/images/01D0AD14-E522-4737-85C9-551BF4FF9D96_jpeg.rf.bff626e61ed25516b6b2de262afd7a19.jpg: 640x640 1 IBUFLASH_MIGRAN, 7.3ms
image 4/57 /content/dataset/test/images/067ACD18-F150-4040-AC28-5E64B7FA4799_jpeg.rf.77ba8b2ed0c9ccd79398fb07e45d2ca9.jpg: 640x640 1 ASPIRINA_100, 7.3ms
image 5/57 /content/dataset/test/images/0C7149D2-80E6-4D03-96D5-066D8308CA15_jpeg.rf.2f0eac5c8877981a5f5cf1c1d87511fb.jpg: 640x640 1 ALKASELTZER, 7.3ms
image 6/57 /content/dataset/test/images/0E7C212C-F026-4668-8C36-E59BDCD8A677_jpeg.rf.e3e551560066c23e43c91c2c80dc6884.jpg: 640x640 1 IBUFLASH_MIGRAN, 7.3ms
image 7/57 /content/dataset/test/images/127E44CB-1D40-4740-B907-90DB3F827E

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'ALKASELTZER', 1: 'ASPIRINA_100', 2: 'BUSCAPINA', 3: 'DOLEX_GRIPA', 4: 'GAVISCON', 5: 'IBUFLASH_MIGRAN', 6: 'IBUPROFENO_GENFAR', 7: 'LEVONORGESTREL', 8: 'NORAVER_GRIPA_DIA', 9: 'VITAC_MK'}
 obb: None
 orig_img: array([[[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        [[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        [[ 97, 163,   4],
         [ 97, 163,   4],
         [ 97, 163,   4],
         ...,
         [103, 163,   3],
         [103, 163,   3],
         [103, 163,   3]],
 
        ...,
 
        [[ 13,  44,   5],
         [ 13,  44,   5],
         [ 13,  44,   5],
         ...,
         [  

In [7]:
#  Export model to tflite


from ultralytics import YOLO

# Load a model
model = YOLO('runs/detect/train/weights/best.pt')  # load a custom trained model

# Export the model
model.export(format='tflite')

Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3007598 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 14, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0', 'onnx2tf>1.17.5,<=1.22.3', 'sng4onnx>=1.0.1', 'onnxslim==0.1.28', 'onnx_graphsurgeon>=0.3.26', 'tflite_support', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 90.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.0/435.0 kB 150.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 297.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 263.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 MB 69.1 MB/

100%|██████████| 1.11M/1.11M [00:00<00:00, 5.69MB/s]
Unzipping calibration_image_sample_data_20x128x128x3_float32.npy.zip to /content/calibration_image_sample_data_20x128x128x3_float32.npy...: 100%|██████████| 1/1 [00:00<00:00, 43.55file/s]


ONNX: starting export with onnx 1.16.1 opset 17...


ONNX: slimming with onnxslim 0.1.28...
ONNX: export success ✅ 1.4s, saved as 'runs/detect/train/weights/best.onnx' (11.8 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.22.3...
TensorFlow SavedModel: export success ✅ 70.6s, saved as 'runs/detect/train/weights/best_saved_model' (29.5 MB)

TensorFlow Lite: starting export with tensorflow 2.15.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'runs/detect/train/weights/best_saved_model/best_float32.tflite' (11.7 MB)

Export complete (72.8s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best_saved_model/best_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train/weights/best_saved_model/best_float32.tflite imgsz=640 data=/content/dataset/data.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best_saved_model/best_float32.tflite'

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Prediction using custom tflite model

#  Prediction using trained model

from ultralytics import YOLO

# Load a pretrained YOLOv8n model
model = YOLO('runs\\detect\\train3\\weights\\best_saved_model\\best_float32.tflite')

# Run inference
model.predict('test_images', save=True, imgsz=640, conf=0.2)

In [1]:
!python --version

Python 3.10.12
